In [31]:
import tabula
import numpy as np
import pandas as pd

In [13]:
file1 = "../doc/BMS.pdf"
table_list = tabula.read_pdf(file1,pages='99-100')

In [25]:
registers_table = (table_list[0].append(table_list[1])).astype('string')
registers_table

C:\Users\dplas\AppData\Local\TEMP_~1/ipykernel_13952/629333286.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  registers_table = (table_list[0].append(table_list[1])).astype('string')


,ADDRESS,POR,NAME,DESCRIPTION
0,0x00,xxxxh,VERSION,Device model and version
1,0x01,0000h,ADDRESS,Device addresses
2,0x02,8000h,STATUS,Status flags
3,0x03,0000h,FMEA1,Failure mode flags 1
4,0x04,0000h,ALRTCELL,Voltage-fault alert flags
5,0x05,0000h,ALRTOVCELL,Overvoltage alert flags
6,0x07,0000h,ALRTUVCELL,Undervoltage alert flags
7,0x08,0000h,ALRTBALSW,Balancing switch alert flags
8,0x0A,0F0Fh,MINMAXCELL,Cell number for the highest and lowest voltage...
9,0x0B,0000h,FMEA2,Failure mode flags 2


In [40]:
cpp_commands = pd.DataFrame({'command':
"#define " + registers_table['NAME'] + "_ADDR " +registers_table['ADDRESS'] + " " + "//" + registers_table['DESCRIPTION'] +" address\n" +
"#define " + registers_table['NAME'] + "_POR " +registers_table['POR'] + " " + "//" + registers_table['DESCRIPTION'] +" power on restart bit"})
cpp_commands


,command
0,#define VERSION_ADDR 0x00 //Device model and v...
1,#define ADDRESS_ADDR 0x01 //Device addresses a...
2,#define STATUS_ADDR 0x02 //Status flags addres...
3,#define FMEA1_ADDR 0x03 //Failure mode flags 1...
4,#define ALRTCELL_ADDR 0x04 //Voltage-fault ale...
5,#define ALRTOVCELL_ADDR 0x05 //Overvoltage ale...
6,#define ALRTUVCELL_ADDR 0x07 //Undervoltage al...
7,#define ALRTBALSW_ADDR 0x08 //Balancing switch...
8,#define MINMAXCELL_ADDR 0x0A //Cell number for...
9,#define FMEA2_ADDR 0x0B //Failure mode flags 2...


In [45]:
with open("MAX17843.h", "w") as BMS_lib:
    for _, row in cpp_commands.iterrows():
        BMS_lib.write(f"{row['command']}\n\n")